### **Data Cleaning Workflow** 
---
#### **1. Import Data**  
- Drop records where:  
   - **`'sii'`** is `NaN`.  
- Save it as target variables:  
   - 🎯 **`sii`**  
---
#### **2. Feature Cleaning**  
-  Drop all **features** that are present **only in the train set**.  
-  Remove features where **`NaN` values ≥ 50%** (Threshold: **`0.5`**).  
-  Find out the `Numerical` and the `Categorical` Features
---
#### **3. Outliers Removal**  
- Find out the 20th `most corraleted` to `sii`features  
   -  Remove the numerical discrete ones
- Remove all the rows with extreme values in **`at least`** one of the `selected features`
---
#### **4. Feature Engineering**  
- **`Missing Value Imputation`** exploiting `Bayesan Ridge`
- **`1HE`** Categorical Features
- Merge **`FGC-Feature`** with its relative **`FGC-Feature_Zone`**
   -  Remove all FGC_Zone and for every record with Zone > 0, merge it with a **`25% more than the original value`**
- Exploit a **`RandomForestClassifier`** and a **`Recursive Feature Elimination`** to filter the best features
- Check the **`HeatMap of the Correlation`** between the features. Remove Features with a correlation between them **≥ 0.9**
---
#### **5. Export Dataset**  
- **Export the cleaned dataset**.  
- Proceed to:  
   - Build **four classification models**.  
   - Perform **hyperparameter tuning**.  
   - Document appropriate **considerations**.

